## How to best aggregate / cluster similar causes and effects and represent them



In [9]:
!pip install vision

Error processing line 1 of /usr/local/lib/python3.6/dist-packages/vision-1.0.0-nspkg.pth:

  Traceback (most recent call last):
    File "/usr/lib/python3.6/site.py", line 174, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 568, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import vision

ModuleNotFoundError: No module named 'vision'

In [31]:
########## Load causes and effects dataset #############

# By default a diabetes dataset is given, but feel free to give your predicted cancer cause-effect set

import pandas as pd
from ast import literal_eval

causes_effects = pd.read_csv("data/diabetes_cause_effect_sentences.csv", sep=";", converters={"tokenized":literal_eval, "io_tags":literal_eval})
print(causes_effects.shape)
causes_effects.head()

In [ ]:
######## Extract causes and effects from io tags #############

def extract_causes_effects(row):
    """ Extracts causes and effects from the io_tags column and saves them in new columns """

    
    if len(row["tokenized"]) != len(row["io_tags"]):
        print("ERROR: Tokenized and tags are not of same length!")
    
    causes = ""
    effects = ""
    last_tag_cause_index = -2
    last_tag_effect_index = -2
    for i, tag in enumerate(row.io_tags):
        if tag == "I-C":
            if i == last_tag_cause_index + 1: # True, if cause consist of several consecutive words
                causes += ","+row["tokenized"][i]
            else: # cause is a new separate cause
                causes += ";"+row["tokenized"][i]
            last_tag_cause_index = i # helps to test if causes and effects consist of CONSECUTIVE words

        elif tag == "I-E":
            if i == last_tag_effect_index + 1: # True, if cause consist of several consecutive words
                effects += ","+row["tokenized"][i]
            else: # cause is a new separate cause
                effects += ";"+row["tokenized"][i]
            last_tag_effect_index = i # helps to test if causes and effects consist of CONSECUTIVE words
    
    if causes.startswith(",") or causes.startswith(";"):
        causes = causes[1:]
    if effects.startswith(",") or effects.startswith(";"):
        effects = effects[1:]        

    
    return pd.Series([row.index, row.text, row.tokenized, row.io_tags, causes, effects]
                     , index=["index", "text", "tokenized", "io_tags", "causes", "effects"])


df_causes_effects = causes_effects.apply(extract_causes_effects, axis=1)
print(df_causes_effects.shape)
df_causes_effects.head()

**Attention:** 
a cause (can consist of multiple words) is separate by "," in the column "causes" and "effect", e.g. type,2,diabetes
if several causes exist in a sentence, then they are separated by ";"

In [ ]:
######## Only consider sentences with both cause AND effect #############

df_new = df_causes_effects[(df_causes_effects.causes.str.len() > 0 ) & (df_causes_effects.effects.str.len() > 0)]
df_new.reset_index(drop=True, inplace=True)
print(df_new.shape)
df_new.head()


In [ ]:
################## SPLIT CAUSES AND EFFECTS s.t. each row has one cause and one effect #######################
df_new = df_new.assign(causes=df_new["causes"].str.split(";")).explode("causes")
df_new = df_new.assign(effects=df_new["effects"].str.split(";")).explode("effects")
df_new.reset_index(drop=True, inplace=True)
print(df_new.shape)
df_new.head(20)

In [ ]:
########## load FastText vectors #####################
from gensim.models.fasttext import FastText
model = FastText.load(path_we)

In [ ]:
########### Load manual diabetes clusters ###########
manual_clusters = pd.read_csv("data/manual_diabetes_clusters.csv", sep=";")
print(manual_clusters.shape)
manual_clusters.head()